# CONUS404 Site Data Selection
Pull CONUS404 data at a set of lat/lon point locations.


In [1]:
import fsspec
import xarray as xr
import hvplot.xarray
import intake
import os
import cartopy.crs as ccrs
import metpy
import warnings
import pandas as pd
import s3fs
import xoak
import dask
import hvplot.pandas # to add .hvplot to DataFrames
from dask.distributed import LocalCluster, Client
warnings.filterwarnings('ignore')

## Open dataset from Intake Catalog
* Select `on-prem` dataset from /caldera if running on prem (Denali/Tallgrass)
* Select `cloud`/`osn` object store data if running elsewhere

In [2]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

['conus404-catalog',
 'conus404-drb-eval-tutorial-catalog',
 'nhm-v1.0-daymet-catalog',
 'nhm-v1.1-c404-bc-catalog',
 'nhm-v1.1-gridmet-catalog',
 'nwis-streamflow-usgs-gages-onprem',
 'nwis-streamflow-usgs-gages-cloud',
 'nwm21-streamflow-usgs-gages-onprem',
 'nwm21-streamflow-usgs-gages-cloud',
 'nwm21-streamflow-cloud',
 'nwm21-scores',
 'lcmap-cloud',
 'rechunking-tutorial-cloud']

In [3]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

['conus404-hourly-onprem',
 'conus404-hourly-cloud',
 'conus404-hourly-osn',
 'conus404-hourly-osn2',
 'conus404-daily-diagnostic-onprem',
 'conus404-daily-diagnostic-cloud',
 'conus404-daily-diagnostic-osn',
 'conus404-daily-onprem',
 'conus404-daily-cloud',
 'conus404-daily-osn',
 'conus404-daily-osn2',
 'conus404-monthly-onprem',
 'conus404-monthly-cloud',
 'conus404-monthly-osn']

In [4]:
## Select the dataset you want to read into your notebook and preview its metadata
dataset = 'conus404-daily-osn' 
cat[dataset]

conus404-daily-osn:
  args:
    consolidated: true
    storage_options:
      anon: true
      client_kwargs:
        endpoint_url: https://renc.osn.xsede.org
      requester_pays: false
    urlpath: s3://rsignellbucket2/hytest/conus404/conus404_daily_202210.zarr
  description: CONUS404 40 years of daily values for subset of model output variables
    derived from hourly values. This dataset is stored on AWS S3 cloud storage in
    a requester-pays bucket.  This dataset is stored on AWS S3 cloud storage in a
    requester-pays bucket. You can work with this data for free in any environment
    (there are no egress fees).
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/subcatalogs


## Parallelize with Dask 
Some of the steps we will take are aware of parallel clustered compute environments
using `dask`. We're going to start a cluster now so that future steps can take advantage
of this ability. 

This is an optional step, but speed ups data loading significantly, especially 
when accessing data from the cloud.

We have documentation on how to start a Dask Cluster in different computing environments [here](../environment_set_up/clusters.md).

In [5]:
#%run ../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
## If this notebook is not being run on Nebari/ESIP, replace the above 
## path name with a helper appropriate to your compute environment.  Examples:
# %run ../environment_set_up/Start_Dask_Cluster_Denali.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_Tallgrass.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_Desktop.ipynb
# %run ../environment_set_up/Start_Dask_Cluster_PangeoCHS.ipynb

## Explore the dataset

In [6]:
ds = cat[dataset].to_dask()

In [7]:
display(ds)

<xarray.Dataset>
Dimensions:         (time: 15336, y: 1015, x: 1367, bottom_top_stag: 51,
                     bottom_top: 50, soil_layers_stag: 4, x_stag: 1368,
                     y_stag: 1016, snow_layers_stag: 3, snso_layers_stag: 7)
Coordinates:
    lat             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lat_u           (y, x_stag) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lat_v           (y_stag, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon_u           (y, x_stag) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon_v           (y_stag, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
  * time            (time) datetime64[ns] 1979-10-01 1979-10-02 ... 2021-09-25
  * x               (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y               (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
Dimensions without coordinates: bottom_top_stag, bottom_top, soil_layers_stag,
                                x_stag, y_stag, snow_layers_stag,
                                snso_layers_stag
Data variables: (12/157)
    ACDEWC          (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ACDRIPR         (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ACDRIPS         (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ACECAN          (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ACEDIR          (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ACETLSM         (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    ...              ...
    ZNU             (bottom_top) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    ZNW             (bottom_top_stag) float32 dask.array<chunksize=(51,), meta=np.ndarray>
    ZS              (soil_layers_stag) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    ZSNSO           (time, snso_layers_stag, y, x) float32 dask.array<chunksize=(36, 7, 350, 350), meta=np.ndarray>
    ZWT             (time, y, x) float32 dask.array<chunksize=(36, 350, 350), meta=np.ndarray>
    crs             int32 ...
Attributes: (12/148)
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    AER_AOD550_VAL:                  0.11999999731779099
    AER_ASY_OPT:                     1
    AER_ASY_VAL:                     0.8999999761581421
    ...                              ...
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    history:                         Tue Mar 29 16:35:22 2022: ncrcat -A -vW ...
    history_of_appended_files:       Tue Mar 29 16:35:22 2022: Appended file ...

In [8]:
# select a variable of interest
var = 'TSK'

In [9]:
ds_var = ds[var]

In [10]:
# get projection info
ds = ds.metpy.parse_cf()
crs = ds[var].metpy.cartopy_crs
#crs

## Read in point data and clean

In [11]:
points_df = pd.read_csv('https://usgs.osn.mghpcc.org/usgspod-testbucket/trends_and_drivers/filtered_temperature_sites.csv')
print(len(points_df))
points_df.head()

2080


,site_id,sources,longitude,latitude
0,Arizona_149,norwest,-112.757795,36.304562
1,Arizona_150,norwest,-112.632052,36.394490
2,Arizona_167,norwest,-111.799499,36.192755
3,Arizona_174,norwest,-112.093022,36.099446
4,BSP-1,ecosheds,-69.080000,46.130000


### Drop rows will null lat, lon

In [12]:
points_df[points_df.isnull().any(axis=1)]

,site_id,sources,longitude,latitude
84,"CTVOLMON-Deep Brook, Unnamed tributary (""""Oil ...",NaN,NaN,NaN
403,NOAA-LFFS - YSI,NaN,NaN,NaN
869,USGS-02172050,NaN,NaN,NaN
872,USGS-021720698,NaN,NaN,NaN
875,USGS-021720869,NaN,NaN,NaN


In [13]:
# drop rows will null lat, lon
points_df.dropna(subset = ['longitude', 'latitude'], inplace=True)
print(len(points_df))

2075


### Set site_id as index

In [14]:
#points_df = points_df.set_index(['site_id', 'longitude', 'latitude'])
points_df = points_df.set_index(['site_id'])
print(len(points_df))

2075


In [15]:
points_df.head()

,sources,longitude,latitude
site_id,,,
Arizona_149,norwest,-112.757795,36.304562
Arizona_150,norwest,-112.632052,36.394490
Arizona_167,norwest,-111.799499,36.192755
Arizona_174,norwest,-112.093022,36.099446
BSP-1,ecosheds,-69.080000,46.130000


### Make sure no site ids are duplicated

In [16]:
points_df[points_df.index.duplicated()]

,sources,longitude,latitude
site_id,,,


### Transform into xarray dataset

In [17]:
points_ds = xr.Dataset.from_dataframe(points_df)

In [18]:
points_ds

<xarray.Dataset>
Dimensions:    (site_id: 2075)
Coordinates:
  * site_id    (site_id) object 'Arizona_149' ... 'WashingtonCoast_98'
Data variables:
    sources    (site_id) object 'norwest' 'norwest' ... 'norwest' 'norwest'
    longitude  (site_id) float64 -112.8 -112.6 -111.8 ... -122.3 -122.2 -122.1
    latitude   (site_id) float64 36.3 36.39 36.19 36.1 ... 47.77 47.8 47.31 47.6

## Find data values at point locations

In [19]:
ds_var.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [20]:
ds_var.xoak.index

In [21]:
#from dask.diagnostics import ProgressBar
#with ProgressBar(), dask.config.set(scheduler='processes'):
ds_selection = ds_var.xoak.sel(lat=points_ds.latitude, lon=points_ds.longitude)

In [22]:
ds_selection

<xarray.DataArray 'TSK' (time: 15336, site_id: 2075)>
dask.array<transpose, shape=(15336, 2075), dtype=float32, chunksize=(36, 2075), chunktype=numpy.ndarray>
Coordinates:
    lat      (site_id) float32 dask.array<chunksize=(2075,), meta=np.ndarray>
    lon      (site_id) float32 dask.array<chunksize=(2075,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1979-10-01 1979-10-02 ... 2021-09-25
    x        (site_id) float64 -1.308e+06 -1.296e+06 ... -1.792e+06 -1.78e+06
    y        (site_id) float64 -1.96e+05 -1.88e+05 ... 1.148e+06 1.176e+06
Dimensions without coordinates: site_id
Attributes:
    description:   SURFACE SKIN TEMPERATURE
    grid_mapping:  crs
    long_name:     Surface skin temperature
    units:         K

## Join selected data back to gage data with site ID, source, lat, lon

In [23]:
ds_selection = xr.merge([points_ds, ds_selection])

## Visualize data to verify results

In [24]:
idx = 300

In [25]:
da = ds_selection.isel(time=idx).load()
df = da.to_dataframe()

In [26]:
df.hvplot.scatter(x='lon', y='lat', c=var, colormap='viridis').opts(clim=(260, 300))

:Scatter   [lon]   (lat)

In [ ]:
da_grid = ds_var.isel(time=idx).load()
da_grid.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM', cmap='viridis').opts('Image', clim=(260, 300))

## Clean up data for saving

In [28]:
# drop CONUS404 grid cell lat/lon, x/y values that data were pulled from, keeping only site's lat/lon to reduce confusion
ds_save = ds_selection.drop_vars(["lat", "lon", "x", "y"])

In [29]:
ds_save

<xarray.Dataset>
Dimensions:    (site_id: 2075, time: 15336)
Coordinates:
  * site_id    (site_id) object 'Arizona_149' ... 'WashingtonCoast_98'
  * time       (time) datetime64[ns] 1979-10-01 1979-10-02 ... 2021-09-25
Data variables:
    sources    (site_id) object 'norwest' 'norwest' ... 'norwest' 'norwest'
    longitude  (site_id) float64 -112.8 -112.6 -111.8 ... -122.3 -122.2 -122.1
    latitude   (site_id) float64 36.3 36.39 36.19 36.1 ... 47.77 47.8 47.31 47.6
    TSK        (time, site_id) float32 dask.array<chunksize=(36, 2075), meta=np.ndarray>

## Save netcdf to OSN pod

In [30]:
fs_write = fsspec.filesystem(
    's3',
    profile='osn-usgs',  ## This is the profile name you configured above.
    client_kwargs={'endpoint_url': 'https://usgs.osn.mghpcc.org/'}
)

In [31]:
fs_write.ls('usgspod-testbucket')
outfile = f'usgspod-testbucket/trends_and_drivers/daily_{var}_at_filtered_temperature_sites.nc'
local_file = f'./daily_{var}_at_filtered_temperature_sites.nc'

Uncomment next two cells to save

In [ ]:
%%time
ds_save.to_netcdf(local_file)

In [ ]:
%%time
fs_write.upload(local_file, outfile)

In [ ]:
fs_write.ls('usgspod-testbucket/trends_and_drivers/')

In [ ]:
print(f'netcdf file size is {fs_write.size(outfile) / 1048576} MB')

### Read back in the nc data to verify it saved correctly

In [ ]:
with fs_write.open(outfile) as f:
    ds_final = xr.open_dataset(f)

In [ ]:
ds_final

## Save csv to OSN pod

In [ ]:
df_save = ds_save.to_dataframe()

In [ ]:
df_save.head()

In [ ]:
csv_outfile = outfile.replace('nc', 'csv')

Uncomment next two cells to save

In [ ]:
# %%time
# with fs_write.open(csv_outfile, mode='wt') as f:
#     df_save.to_csv(f)

In [ ]:
fs_write.ls('usgspod-testbucket/trends_and_drivers/')

In [ ]:
print(f'csv file size is {fs_write.size(csv_outfile) / 1048576} MB')

### Read back in the csv data to verify it saved correctly

In [ ]:
with fs_write.open(csv_outfile) as f:
    df_final_csv = pd.read_csv(f)

In [ ]:
df_final_csv.head()

## Shutdown cluster

In [ ]:
#client.close(); cluster.shutdown()